In [0]:
spark.conf.set("fs.azure.account.key.cs0st0rage.dfs.core.windows.net", dbutils.secrets.get(scope = 'az_cs_scope', key= 'acc-key'))

delta_table_path_final = "abfss://silver@cs0st0rage.dfs.core.windows.net/"

df = spark.read.format("delta").load(delta_table_path_final)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

# Define window partitioned by Restaurant_ID and ordered by Inserted_Time
windowSpec = Window.partitionBy("Restaurant_ID").orderBy(col("timestamp").desc())

# Assign row numbers to each record for each Restaurant_ID based on the Inserted_Time
df_with_rn = df.withColumn("row_number", row_number().over(windowSpec))

# Filter only the latest record for each Restaurant_ID (row_number = 1)
df_latest = df_with_rn.filter(col("row_number") == 1).drop("row_number")

# Show the result
# df_latest.display()


In [0]:
delta_table_path_final = "abfss://gold@cs0st0rage.dfs.core.windows.net/"
df_latest.write.format("delta").mode("append").save(delta_table_path_final)